# Hello 3D Point Cloud to Mesh

In [1]:
# Prerequisites and Dependencies
import numpy as np
import open3d as o3d  



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Load Point Cloud to Numpy Array

In [8]:
file_path = "point_clouds_synthetic/duck_rainbow.xyzrgb"

pc = np.loadtxt(file_path)
print("Shape of numpy array: ", pc.shape)

Shape of numpy array:  (5100, 6)


### Transform from Numpy to Open3D

In [10]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc[:,:3]) # XYZ
pcd.colors = o3d.utility.Vector3dVector(pc[:,3:6]/255) # RGB
# pcd.normals = o3d.utility.Vector3dVector(pc[:,6:9]) # Normals


### Visualize

In [11]:
o3d.visualization.draw_geometries([pcd])

### Mesh with Ball-Pivoting Algorithm (BPA)

In [14]:
# Compute Average Distances
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist


In [15]:
# Estimate Normals
# Use a neighborhood roughly a few times the average spacing
normal_radius = 2.5 * avg_dist
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(
        radius=normal_radius, max_nn=30
    )
)

In [16]:
# Orient normals consistently to avoid random flips
pcd.orient_normals_consistent_tangent_plane(k=24)

In [17]:
# Ball Pivoting 
# Choose a set of radii around the point spacing
bpa_base = 3.0 * avg_dist
radii = o3d.utility.DoubleVector([bpa_base, 2*bpa_base, 4*bpa_base])

bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, radii
)

In [18]:
# Post process
bpa_mesh.remove_degenerate_triangles()
bpa_mesh.remove_duplicated_triangles()
bpa_mesh.remove_duplicated_vertices()
bpa_mesh.remove_non_manifold_edges()
bpa_mesh.compute_vertex_normals()

TriangleMesh with 5100 points and 7024 triangles.

In [19]:
# Visualize
o3d.visualization.draw_geometries(
    [pcd, bpa_mesh],
    window_name="Ball Pivoting Surface Reconstruction",
    width=1280,
    height=720,
    mesh_show_back_face=True  # makes the mesh visible from both sides
)

### Mesh with Poisson Reconstruction

In [20]:
# Estimate & orient normals (Poisson also requires normals)
# Use a neighborhood scaled to your data
dists = pcd.compute_nearest_neighbor_distance()
avg = float(np.mean(dists))
search_radius = 2.5 * avg if np.isfinite(avg) and avg > 0 else 0.02  # fallback

pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=search_radius, max_nn=30)
)
pcd.orient_normals_consistent_tangent_plane(k=24)

In [26]:
# Poisson reconstruction
# depth controls octree resolution (higher = more detail, slower)
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    pcd, depth=9, scale=1.1, linear_fit=True
)

In [27]:
# Trim low-density vertices to reduce spurious surfaces
densities = np.asarray(densities)
# Keep the densest 98% (tune this: 0.98 keeps more, 0.95 trims more)
density_thresh = np.quantile(densities, 0.02)
to_remove = densities < density_thresh

# Sanity check lengths match BEFORE removing
assert len(to_remove) == np.asarray(mesh.vertices).shape[0], \
    f"Mask {len(to_remove)} vs verts {np.asarray(mesh.vertices).shape[0]} mismatch"

mesh.remove_vertices_by_mask(to_remove)

In [28]:
# Crop to remove far-away artifacts
bbox = pcd.get_axis_aligned_bounding_box()
mesh = mesh.crop(bbox)

In [29]:
# Clean up & compute vertex normals for nice shading
mesh.remove_degenerate_triangles()
mesh.remove_duplicated_triangles()
mesh.remove_duplicated_vertices()
mesh.remove_non_manifold_edges()
mesh.compute_vertex_normals()

TriangleMesh with 19353 points and 38431 triangles.

In [30]:
# Visualize
# Paint mesh and point cloud different colors for contrast
if pcd.has_colors():
    # keep original pcd colors
    pass
else:
    pcd.paint_uniform_color([0.1, 0.6, 1.0])  # cyan-ish

mesh.paint_uniform_color([0.8, 0.8, 0.8])  # light gray

o3d.visualization.draw_geometries(
    [pcd, mesh],
    window_name="Poisson Reconstruction (Point Cloud + Mesh)",
    width=1280,
    height=720,
    mesh_show_back_face=True
)